In [3]:
import json
import os

import pandas as pd

In [2]:
phi2 = "../results/character/phi-2/"
phi3 = "../results/character/phi-3/"

In [28]:
def generate_questioner(dir, n, fn):
    files = os.listdir(dir)
    files = [f for f in files if f.endswith(".json")]

    hp = list(filter(lambda x: "hp" in x, files))
    asoif = list(filter(lambda x: "asoif" in x, files))

    hp_questions = []

    for f in hp:
        character = f.split("_")[1]
        with open(dir + f, "r") as file:
            data = json.load(file)
            for k, v in data.items():
                hp_questions.append(
                    {
                        "source": "hp",
                        "character": character,
                        "question": k,
                        "A": v["with_rag_reveal"],
                        "B": v["without_rag"],
                        "C": v["with_rag_hide"],
                    }
                )

    asoif_questions = []

    for f in asoif:
        character = f.split("_")[1]
        with open(dir + f, "r") as file:
            data = json.load(file)

            for k, v in data.items():
                asoif_questions.append(
                    {
                        "source": "asoif",
                        "character": character,
                        "question": k,
                        "A": v["with_rag_reveal"],
                        "B": v["without_rag"],
                        "C": v["with_rag_hide"],
                    }
                )

    df_hp = pd.DataFrame(hp_questions)
    df_asoif = pd.DataFrame(asoif_questions)

    hp_sampled = df_hp.sample(n)
    asoif_sampled = df_asoif.sample(n)

    # concat and shuffle
    df = pd.concat([hp_sampled, asoif_sampled])
    df = df.sample(frac=1)

    df["rank"] = ""

    df.to_csv(fn, index=False)


for i in ["jan", "ziga", "matej"]:
    generate_questioner(phi2, 30, f"../results/questioners/phi-2-{i}.csv")
    generate_questioner(phi3, 30, f"../results/questioners/phi-3-{i}.csv")